In [70]:
import time
import pandas as pd
import warnings
from selenium import webdriver
from time import sleep
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [72]:
warnings.filterwarnings('ignore', category=FutureWarning) 

In [62]:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"

In [63]:
url = "https://www.linkedin.com/in/gaurang-gujrati-088a931b8/"

In [64]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
chrome_options.add_argument(f'user_agent={user_agent}')
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-infobars")
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)

In [42]:

connections_link_element = driver.find_element(By.XPATH, '//a[contains(@href, "/search/results/people/?network")]')
connections_link = connections_link_element.get_attribute("href")


In [43]:
connections_link

'https://www.linkedin.com/search/results/people/?network=%5B%22F%22%5D&origin=MEMBER_PROFILE_CANNED_SEARCH'

In [44]:
driver.get(connections_link)

In [53]:
profile_links = []

In [ ]:
while True:
    profile_elements = driver.find_elements(By.XPATH, '//a[contains(@href, "miniProfile")]')
    profile_links += [el.get_attribute('href') for el in profile_elements]
    
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    try:
        next_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'artdeco-pagination__button--next'))
        )
        next_button.click()
        time.sleep(2)
    except TimeoutException:
        break

In [65]:
profile_links = list(set(profile_links))

In [66]:
profile_links

['https://www.linkedin.com/in/jee-sh-b94a3626a?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAEIBOiABHARz3Gk_4C33xrGvAiM1ppjQLi0',
 'https://www.linkedin.com/in/sudipta-sundar-pal-bb2aa81b6?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADJZ_rYBU-7bRHC_iC1LITg_L9z16dU_LJA',
 'https://www.linkedin.com/in/ACoAAC8lj70BmXNpKwoC-bUSOptxFP4K45j1yoI?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAC8lj70BmXNpKwoC-bUSOptxFP4K45j1yoI',
 'https://www.linkedin.com/in/rudransh-pandey-7675171b8?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADKfQH8BvcrITAo_goHXH0RjgXKwzmXqkA0',
 'https://www.linkedin.com/in/shrijan-prakash-a71111169?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACgPubUBfrwZX5UFPsL9Jd2BzgUvfcTaqKw',
 'https://www.linkedin.com/in/shinjan-patra-82b2bb21b?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADdlUeMBdSn308lkMh5f-NCX4275_IUX-7w',
 'https://www.linkedin.com/in/abhijith-hari-17a581217?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADa5O64B9sAEjZasdVbwTWvAidGTiORD4tU',
 'https:/

In [67]:
len(profile_links)

985

In [58]:
df = pd.DataFrame(profile_links, columns=['Connection Links'])

In [59]:
df.to_excel('connection_links.xlsx', index=False)

In [ ]:
df = pd.read_excel('connection_links.xlsx')

results = pd.DataFrame(columns=['Connection Links', 'Connection Name', 'Premium'])

for link in df['Connection Links']:
    driver.get(link)
    
    # profile name
    profile_name_element = driver.find_element(By.TAG_NAME, 'h1')
    profile_name = profile_name_element.text

    try:
        driver.find_element(By.XPATH, '//span[@class="pv-member-badge pv-member-badge--for-top-card"]')
        premium = "Yes"
    except NoSuchElementException:
        premium = "No"

    results = results.append({
        'Connection Links': link,
        'Connection Name': profile_name,
        'Premium': premium
    }, ignore_index=True)

    # 5 sec wait time to avoid getting blocked
    time.sleep(5)


In [92]:
results.to_excel('connection_info.xlsx', index=False)